Custom functions

Line Typology Analysis and Update

In [ ]:
# @title
# Function to count the occurrence of each typology in the document
def count_line_typologies(doc_pk):
    # Dictionary to store the count of each typology
    typology_counts = defaultdict(int)

    # Get all parts of the document
    parts = get_all_parts(doc_pk)

    for part in parts:
        part_pk = part['pk']
        #print(f"Processing part {part_pk}")

        # Fetch all lines in the current part
        lines = get_all_lines_of_part(doc_pk, part_pk)

        for line in lines:
            typology = line.get('typology', None)

            # If the line has a typology, count it
            if typology:
                typology_counts[typology] += 1
            else:
                typology_counts['None'] += 1  # Count lines with no typology as 'None'

    # Convert typology counts to a normal dictionary (optional, for easier reading)
    return dict(typology_counts)

# Call the function with the document primary key and print the results
typology_summary = count_line_typologies(doc_pk=8)  # Replace XXX with the actual document primary key

# Print the result
for typology, count in typology_summary.items():
    print(f"Typology '{typology}': {count} lines")


Typology '9': 834 lines
Typology '20': 133 lines
Typology '27': 30 lines
Typology 'None': 1 lines


In [ ]:
# @title
# Function to identify lines with specific typologies (None or 18)
def find_specific_typologies(doc_pk, target_typologies):
    # List to store lines with matching typologies
    matching_lines = []

    # Get all parts of the document
    parts = get_all_parts(doc_pk)

    for part in parts:
        part_pk = part['pk']
        #print(f"Processing part {part_pk}")

        # Fetch all lines in the current part
        lines = get_all_lines_of_part(doc_pk, part_pk)

        for line in lines:
            typology = line.get('typology', None)

            # Check if the typology is in our target list
            if typology in target_typologies or typology is None:
                # Collect detailed information about the line
                line_info = {
                    'line_pk': line['pk'],
                    'document_part': line['document_part'],
                    'typology': typology,
                    'content': None  # Default None for content
                }

                # Add the line info to the matching list
                matching_lines.append(line_info)

    return matching_lines

# Define the target typologies to search for
target_typologies = [18]  # You can add more typologies if needed

# Call the function to find lines with typology None or 18
specific_lines = find_specific_typologies(doc_pk=8, target_typologies=target_typologies)  # Replace XXX with your document primary key

# Print out the details of the lines found
for line in specific_lines:
    typology_label = 'None' if line['typology'] is None else line['typology']
    print(f"Line PK: {line['line_pk']}, Document Part: {line['document_part']}, Typology: {typology_label}, Content: {line['content']}")


Line PK: 10740, Document Part: 255, Typology: None, Content: None


In [ ]:
#change typology of specific line

# Set your document primary key, part primary key, and line primary key
doc_pk = 8
part_pk = 291
line_pk = 10853

# You already know that the 'default' typology has a PK of 9
default_typology_pk = 9

# Fetch the specific line data (assuming the get_line function exists)
line_data = get_line(doc_pk, part_pk, line_pk)

# Check if typology is None and update it, change according to need
if line_data['typology'] is None:
    line_data['typology'] = default_typology_pk  # Set typology to 'default' PK

    # Prepare the payload with the required fields
    update_payload = {
        'typology': line_data['typology'],  # Use the PK for 'default'
        'document_part': line_data['document_part']  # Include the document part PK
    }

    # Use the update_line function to update this specific line
    update_url = get_specific_line_url(doc_pk, part_pk, line_pk)  # Generate the specific URL for the line
    result = update_item(update_url, update_payload)  # Send the update

    # Check the response
    if result.status_code == 200:
        print(f"Line {line_pk} typology updated to 'default'")
    else:
        print(f"Failed to update line {line_pk}, response: {result.status_code}, {result.content}")
else:
    print(f"Line {line_pk} already has typology: {line_data['typology']}")


Line 10853 typology updated to 'default'


In [ ]:
#Bulk change the typology of all lines with a specific name

# Define the default typology PK
default_typology_pk = 9  # 'default' typology has a PK of 9

# Function to process all lines in a document and update typology if it's None
def process_document_lines_bulk(doc_pk):
    # Get all parts of the document
    parts = get_all_parts(doc_pk)  # Assuming this function exists to fetch all parts of the document

    for part in parts:
        part_pk = part['pk']
        print(f"Processing part {part_pk}")

        # Fetch all lines in the current part
        lines = get_all_lines_of_part(doc_pk, part_pk)  # Assuming this function exists to fetch all lines for the part
        updated_lines = []  # To store the lines that need updating

        for line in lines:
            line_pk = line['pk']

            # Check if the line's typology is None
            if line['typology'] is None:
                # Update the line's typology to 'default'
                line['typology'] = default_typology_pk  # Set to 'default' PK

                # Prepare the updated line data with required fields
                updated_lines.append({
                    'pk': line_pk,
                    'typology': line['typology'],
                    'document_part': line['document_part']
                })
            else:
                print(f"Line {line_pk} in part {part_pk} already has typology: {line['typology']}")

        # If we have lines to update, send them in bulk
        if updated_lines:
            result = bulk_update_lines(doc_pk, part_pk, updated_lines)
            if result.status_code == 200:
                print(f"Successfully updated {len(updated_lines)} lines in part {part_pk}")
            else:
                print(f"Failed to update lines in part {part_pk}, response: {result.status_code}, {result.content}")
        else:
            print(f"No lines to update in part {part_pk}")

# Call the function with the document primary key
process_document_lines_bulk(doc_pk=8) #

Processing part 241
Successfully updated 4 lines in part 241
Processing part 242
Line 6223 in part 242 already has typology: 20
Line 6224 in part 242 already has typology: 20
Line 6372 in part 242 already has typology: 20
Line 6226 in part 242 already has typology: 27
No lines to update in part 242
Processing part 243
Successfully updated 4 lines in part 243
Processing part 244
Successfully updated 3 lines in part 244
Processing part 245
Line 6238 in part 245 already has typology: 9
Successfully updated 2 lines in part 245
Processing part 246
Successfully updated 4 lines in part 246
Processing part 247
Line 6249 in part 247 already has typology: 9
Line 6253 in part 247 already has typology: 20
Line 6254 in part 247 already has typology: 20
Successfully updated 2 lines in part 247
Processing part 248
Line 6256 in part 248 already has typology: 20
Line 6257 in part 248 already has typology: 20
Line 6258 in part 248 already has typology: 20
Line 6259 in part 248 already has typology: 20
L

Repolygonization Games

Functions that simplify baselines, align them horizontally and contain them within their regions, and then repolygonize, with custom values per line type

In [ ]:
from shapely.geometry import LineString, Polygon

# Function to contain lines within region boundaries and track adjusted lines
def contain_lines_within_region_boundaries(doc_pk, part_pk):
    # Fetch page height and width
    page_height, page_width = get_page_height_width(doc_pk, part_pk)

    # Fetch all regions and lines for the part
    regions = get_all_regions_of_part(doc_pk, part_pk)
    lines = get_all_lines_of_part(doc_pk, part_pk)

    adjusted_lines = []

    # Create a circumference (boundary) for each region
    for region in regions:
        region['Circumference'] = Polygon(region['box'])

    # Iterate through each line
    for line in lines:
        baseline = line['baseline']

        # Ensure the baseline is correctly oriented
        if baseline[0][0] > baseline[-1][0]:
            baseline.reverse()

        # Find the corresponding region for this line
        this_region = search(line['region'], regions)
        region_boundary = this_region['Circumference']

        # Create a LineString for the baseline
        baseline_line = LineString(baseline)

        # If the baseline extends beyond the region, clip it to the boundary
        if not region_boundary.contains(baseline_line):
            # Find the intersection between the baseline and region boundary
            intersection_line = region_boundary.intersection(baseline_line)

            if not intersection_line.is_empty:
                # Extract the new baseline from the intersection points
                if isinstance(intersection_line, LineString):
                    new_baseline = [[int(round(coord[0])), int(round(coord[1]))] for coord in intersection_line.coords]
                else:
                    # Handle cases where the intersection is a collection of points/lines
                    new_baseline = [[int(round(coord[0])), int(round(coord[1]))] for geom in intersection_line.geoms for coord in geom.coords]

                # Reverse baseline if originally reversed
                if baseline[0][0] > baseline[-1][0]:
                    new_baseline.reverse()

                # Add the adjusted line to the list
                adjusted_lines.append(line['pk'])
                new_line = {'pk': line['pk'], 'baseline': new_baseline}

                # Bulk update the lines with the new baselines
                bulk_update_lines(doc_pk, part_pk, [new_line])

    return adjusted_lines

# Main function to contain lines, then apply custom repolygonization based on line typology
def contain_and_repolygonize_with_custom_ascenders_descenders(doc_pk, part_pk, headers):
    # Step 1: Contain lines within region boundaries and track adjusted lines
    adjusted_lines = contain_lines_within_region_boundaries(doc_pk, part_pk)

    # Step 2: Apply custom repolygonization for specific line typologies (only for adjusted lines)
    if adjusted_lines:
        # Repolygonize for line type 9
        create_normalized_line_polygons_with_fixed_ascenders_descenders_4line_and_regiontypology(
            doc_pk=doc_pk, part_pk=part_pk, method='fix', linetypelist=[9],
            ascender=30, descender=45, safetydistance=3
        )

        # Repolygonize for line types 20 and 27
        create_normalized_line_polygons_with_fixed_ascenders_descenders_4line_and_regiontypology(
            doc_pk=doc_pk, part_pk=part_pk, method='fix', linetypelist=[20, 27],
            ascender=65, descender=60, safetydistance=3
        )

        print(f"Repoygonized {len(adjusted_lines)} lines based on custom ascender and descender values.")
    else:
        print("No lines were out of bounds or needed repolygonization.")

# Example usage
doc_pk = 8  # Replace with actual document primary key
part_pk = 393  # Replace with actual part primary key

# Call the function to contain and repolygonize lines
contain_and_repolygonize_with_custom_ascenders_descenders(doc_pk, part_pk, headers)


In [ ]:

from shapely.geometry import LineString, Polygon
import requests
import json

# Function to gently align handwritten baselines without flattening them too much
def smooth_handwritten_baseline(baseline):
    # Calculate a gentle adjustment for handwritten lines: average y, but retain some slope
    smoothed_baseline = []
    x_coords = [point[0] for point in baseline]
    y_coords = [point[1] for point in baseline]

    # Fit a simple linear regression (y = mx + b) to preserve some slope
    if len(x_coords) > 1:
        m = (y_coords[-1] - y_coords[0]) / (x_coords[-1] - x_coords[0])  # Slope
        b = y_coords[0] - m * x_coords[0]  # Intercept
        # Generate smoothed baseline points
        smoothed_baseline = [[x, int(round(m * x + b))] for x in x_coords]
    else:
        # If only one point, keep the original baseline
        smoothed_baseline = baseline

    return smoothed_baseline

# Function to make the baseline more horizontally aligned
def make_baseline_horizontal(baseline):
    # Calculate the average y-coordinate of the baseline points
    avg_y = sum([point[1] for point in baseline]) / len(baseline)

    # Adjust the y-coordinates of the baseline to be closer to the average
    horizontal_baseline = [[x, int(round(avg_y))] for x, y in baseline]

    return horizontal_baseline

# Function to simplify, align, contain, and repolygonize lines according to their typology
def simplify_align_contain_and_repolygonize_lines(doc_pk, part_pk, simplification=10):
    # Fetch the URL for lines in the part
    lines_url = get_lines_url(doc_pk, part_pk)

    # Fetch all lines and regions in the part
    lines = loop_through_itempages_and_get_all(lines_url)
    regions = get_all_regions_of_part(doc_pk, part_pk)

    # Create polygons for each region (boundary)
    for region in regions:
        region['Circumference'] = Polygon(region['box'])

    # List to track lines that need repolygonization by typology
    repolygonize_lines_by_type = {
        9: [],      # Line type 9
        20: [],     # Line type 20
        27: []      # Line type 27
    }

    # Iterate through each line for simplification, alignment, containment, and repolygonization
    for line in lines:
        baseline = line['baseline']
        baseline_line = LineString(baseline)  # Convert the baseline to LineString object

        # Step 1: Simplify the baseline
        simplified_baseline = baseline_line.simplify(simplification)
        new_baseline = [[int(round(x)), int(round(y))] for x, y in simplified_baseline.coords]

        # Step 2: Align the baseline based on typology
        typology = line.get('typology', None)
        if typology in [20, 27]:
            # For handwritten lines (20, 27), apply gentle smoothing, retaining slope
            aligned_baseline = smooth_handwritten_baseline(new_baseline)
        else:
            # For other lines, flatten more aggressively
            aligned_baseline = make_baseline_horizontal(new_baseline)

        # Step 3: Contain the baseline within the region if needed
        region = next((r for r in regions if r['pk'] == line['region']), None)
        if region:
            region_boundary = region['Circumference']
            baseline_line = LineString(aligned_baseline)  # Update baseline to horizontally aligned one

            if not region_boundary.contains(baseline_line):
                # If baseline extends beyond the region, intersect it with the region boundary
                intersection_line = region_boundary.intersection(baseline_line)

                if not intersection_line.is_empty:
                    if isinstance(intersection_line, LineString):
                        aligned_baseline = [[int(round(x)), int(round(y))] for x, y in intersection_line.coords]
                    else:
                        # Handle cases where intersection might be a collection of points
                        aligned_baseline = [[int(round(x)), int(round(y))] for geom in intersection_line.geoms for x, y in geom.coords]

        # Step 4: Send updated baseline to the API
        line_url = lines_url + str(line['pk']) + '/'
        line_json = {'baseline': aligned_baseline}
        res_line = requests.patch(line_url, headers=headers, data=json.dumps(line_json))

        if res_line.status_code == 200:
            #print(f"Simplified, aligned, and contained baseline for line {line['pk']} updated successfully.")

            # After baseline is simplified, aligned, and contained, check the typology for custom repolygonization
            if typology in repolygonize_lines_by_type:
                repolygonize_lines_by_type[typology].append(line['pk'])
        else:
            print(f"Failed to update line {line['pk']}. Status: {res_line.status_code}")

    # Apply custom repolygonization based on line typology
    # Repolygonize for line type 9
    if repolygonize_lines_by_type[9]:
        create_normalized_line_polygons_with_fixed_ascenders_descenders_4line_and_regiontypology(
            doc_pk=doc_pk, part_pk=part_pk, method='fix', linetypelist=[9],
            ascender=30, descender=45, safetydistance=3
        )
        #print(f"Repoygonized {len(repolygonize_lines_by_type[9])} lines of type 9.")

    # Repolygonize for line types 20 and 27
    if repolygonize_lines_by_type[20] or repolygonize_lines_by_type[27]:
        create_normalized_line_polygons_with_fixed_ascenders_descenders_4line_and_regiontypology(
            doc_pk=doc_pk, part_pk=part_pk, method='fix', linetypelist=[20, 27],
            ascender=65, descender=60, safetydistance=3
        )
        #print(f"Repoygonized {len(repolygonize_lines_by_type[20]) + len(repolygonize_lines_by_type[27])} lines of types 20 and 27.")

# Example usage
doc_pk = 8  # Replace with actual document primary key
part_pk = 393  # Replace with actual part primary key
simplification_tolerance = 20  # Adjust simplification tolerance as needed
simplify_align_contain_and_repolygonize_lines(doc_pk, part_pk, simplification=simplification_tolerance)


Simplified, aligned, and contained baseline for line 6316 updated successfully.
Simplified, aligned, and contained baseline for line 6317 updated successfully.
Simplified, aligned, and contained baseline for line 6318 updated successfully.
Simplified, aligned, and contained baseline for line 6319 updated successfully.
Simplified, aligned, and contained baseline for line 6320 updated successfully.
Repoygonized 5 lines of types 20 and 27.


In [ ]:
doc_pk = 8
parts = get_all_parts(doc_pk)

for part in parts:
    part_pk = part['pk']
    simplify_align_contain_and_repolygonize_lines(doc_pk, part_pk, simplification=15)


line association and repolygonization

In [ ]:
# check whether ther are any non-associated lines
doc_pk = 8
parts = get_all_parts(doc_pk)

for part in parts:
    part_pk = part['pk']
    find_or_delete_unlinked_lines(doc_pk, part_pk, do_delete=False)

In [ ]:
# using Daniel's functions of association and repolygonization
part_pk = 389
associate_lines_with_existing_regions_and_reorder(doc_pk,part_pk)
create_normalized_line_polygons_with_fixed_ascenders_descenders_4line_and_regiontypology(
    8,part_pk,method='fix', regiontypelist=[],linetypelist=[9,20],ascender=30,descender=45, safetydistance=3)

389
associating lines
[{'pk': 18849, 'document_part': 389, 'external_id': 'eSc_line_843c3d86', 'order': 4, 'region': 5487, 'baseline': [[784, 486], [1112, 486]], 'mask': [[783, 483], [782, 489], [799, 490], [808, 499], [817, 496], [824, 501], [848, 501], [859, 490], [895, 497], [904, 491], [916, 491], [925, 501], [949, 501], [960, 492], [973, 492], [983, 501], [997, 494], [1003, 501], [1020, 501], [1029, 494], [1037, 501], [1049, 496], [1054, 501], [1078, 497], [1099, 501], [1110, 492], [1111, 476], [1102, 470], [1053, 470], [1050, 473], [1044, 470], [1026, 473], [1003, 469], [1000, 473], [995, 469], [981, 469], [974, 476], [953, 476], [934, 469], [922, 475], [843, 475], [833, 471], [825, 475], [783, 475], [783, 483]], 'typology': 9}]
reordering
reorder  389


<Response [200]>

Region Typology Analysis

In [ ]:
# Function to count the occurrence of each region typology in the document
def count_region_typologies(doc_pk):
    # Dictionary to store the count of each typology
    region_typology_counts = defaultdict(int)

    # Get all parts of the document
    parts = get_all_parts(doc_pk)

    for part in parts:
        part_pk = part['pk']
        #print(f"Processing part {part_pk}")

        # Fetch all regions in the current part
        regions = get_all_regions_of_part(doc_pk, part_pk)  # Assuming this function fetches all regions for the part

        for region in regions:
            region_typology = region.get('typology', None)

            # If the region has a typology, count it
            if region_typology:
                region_typology_counts[region_typology] += 1
            else:
                region_typology_counts['None'] += 1  # Count regions with no typology as 'None'

    # Convert the region typology counts to a normal dictionary (optional, for easier reading)
    return dict(region_typology_counts)

# Call the function with the document primary key and print the results
region_typology_summary = count_region_typologies(doc_pk=555)  # Replace XXX with the actual document primary key

# Print the result
for typology, count in region_typology_summary.items():
    print(f"Typology '{typology}': {count} regions")


Typology '100': 207 regions
Typology '101': 40 regions
Typology '104': 455 regions
Typology '102': 100 regions
Typology '103': 43 regions
Typology '105': 36 regions
Typology '106': 32 regions
Typology '107': 1 regions


In [ ]:
# Function to find regions with specific typologies (37, 58, None)
def find_specific_region_typologies(doc_pk, target_typologies):
    # List to store regions with matching typologies
    matching_regions = []

    # Get all parts of the document
    parts = get_all_parts(doc_pk)

    for part in parts:
        part_pk = part['pk']
        #print(f"Processing part {part_pk}")

        # Fetch all regions in the current part
        regions = get_all_regions_of_part(doc_pk, part_pk)

        for region in regions:
            region_typology = region.get('typology', None)

            # Check if the region's typology is in our target list
            if region_typology in target_typologies or region_typology is None:
                # Collect detailed information about the region
                region_info = {
                    'region_pk': region['pk'],
                    'document_part': part_pk,
                    'typology': region_typology,
                    #'external_id': region.get('external_id', 'No ID'),
                    #'box': region.get('box', 'No box info')
                }

                # Add the region info to the matching list
                matching_regions.append(region_info)

    return matching_regions

# Define the target typologies to search for
target_typologies = [107]  # Adding None as a separate case

# Call the function to find regions with typology 37, 58, or None
specific_regions = find_specific_region_typologies(doc_pk=555, target_typologies=target_typologies)  # Replace XXX with the actual document primary key

# Print out the details of the regions found
if specific_regions:
    print("Found regions with specific typologies:")
    for region in specific_regions:
        typology_label = 'None' if region['typology'] is None else region['typology']
        print(f"Region PK: {region['region_pk']}, Document Part: {region['document_part']}, Typology: {typology_label}") #External ID: {region['external_id']}, Box: {region['box']}")
else:
    print("No regions found with the specified typologies.")


Found regions with specific typologies:
Region PK: 7589, Document Part: 5739, Typology: 107


In [ ]:
delete_region(8,241,2638)

<Response [204]>

In [ ]:
# Function to check for duplicate region types within each part of the document
def check_duplicate_region_typologies(doc_pk):
    # Get all parts of the document
    parts = get_all_parts(doc_pk)

    # List to store parts with duplicate region types
    parts_with_duplicates = []

    for part in parts:
        part_pk = part['pk']
        # Dictionary to store the count of each typology in the part
        region_typology_counts = defaultdict(int)

        # Fetch all regions in the current part
        regions = get_all_regions_of_part(doc_pk, part_pk)  # Assuming this function fetches all regions for the part

        # Count the occurrence of each region typology in the part
        for region in regions:
            region_typology = region.get('typology', None)
            if region_typology:
                region_typology_counts[region_typology] += 1
            else:
                region_typology_counts['None'] += 1  # Count regions with no typology as 'None'

        # Check for duplicates (i.e., any region type with a count > 1)
        duplicate_typologies = [typology for typology, count in region_typology_counts.items() if count > 1]

        # If duplicates are found, record the part PK and the duplicate types
        if duplicate_typologies:
            parts_with_duplicates.append({
                'part_pk': part_pk,
                'duplicate_typologies': duplicate_typologies
            })

    # Report parts with duplicate region types
    if parts_with_duplicates:
        print("Found duplicate region types in the following parts:")
        for part_info in parts_with_duplicates:
            print(f"Part PK: {part_info['part_pk']}, Duplicate Region Types: {part_info['duplicate_typologies']}")
    else:
        print("No duplicate region types found in any part.")

# Example usage
doc_pk = 8  # Replace with your actual document primary key
check_duplicate_region_typologies(doc_pk)


No duplicate region types found in any part.


In [ ]:
delete_region(8,292,3367)

<Response [204]>

In [ ]:
# Function to identify empty lines across all parts of the document
def identify_empty_lines_in_document(doc_pk, tr_level, min_length=1):
    # Fetch all parts of the document
    parts = get_all_parts(doc_pk)

    # Iterate over each part
    for part in parts:
        part_pk = part['pk']
        #print(f"Processing part {part_pk}...")

        # Fetch transcriptions and all lines for the part
        transcriptions = get_page_transcription(doc_pk, part_pk, tr_level)
        lines = get_all_lines_of_part(doc_pk, part_pk)
        line_pks = get_pks_of_dict_list(lines)

        # List to store lines that have empty or short transcriptions
        lines2identify = list()

        # Identify lines with short transcription content
        for tr in transcriptions:
            if len(tr['content'].strip()) < min_length:  # Ensure spaces are counted as empty
                lines2identify.append(tr['line'])  # Store line pk

        # Identify lines that have no transcriptions
        for line_pk in line_pks:
            if line_pk not in [tr['line'] for tr in transcriptions]:  # Lines with no transcription
                lines2identify.append(line_pk)

        # Report the identified lines instead of deleting them
        if len(lines2identify) > 0:
            print(f"Identified {len(lines2identify)} lines with empty or no transcription in part {part_pk}:")
            for line_pk in lines2identify:
                print(f"Line PK: {line_pk}")

# Example usage for document pk 8
doc_pk = 8  # Document primary key
tr_level = 18  # Replace with actual transcription level pk (e.g., 'manual' = 18')
min_length = 1  # Minimum length to consider a transcription not empty

# Identify empty lines across all parts of the document
identify_empty_lines_in_document(doc_pk, tr_level, min_length=min_length)


Identified 1 lines with empty or no transcription in part 245:
Line PK: 23301


In [ ]:
def find_parts_without_regions(doc_pk):
    """
    Find and report parts of a document that have no regions at all.

    Parameters:
    - doc_pk: Primary key of the document to check.
    """
    # Fetch all parts of the document
    parts = get_all_parts(doc_pk)

    # List to track parts without regions
    parts_without_regions = []

    # Iterate through each part and check for regions
    for part in parts:
        part_pk = part['pk']

        # Fetch regions for the part
        regions = get_all_regions_of_part(doc_pk, part_pk)

        # If no regions are found, add the part to the report list
        if not regions:
            parts_without_regions.append(part_pk)

    # Report the parts that have no regions
    if parts_without_regions:
        print("Parts without regions found:")
        for part_pk in parts_without_regions:
            print(f"Part PK: {part_pk}")
    else:
        print("All parts have regions.")
    return parts_without_regions

# Example usage
doc_pk = 555  # Replace with actual document primary key

parts_without_regions  = find_parts_without_regions(doc_pk)


Parts without regions found:
Part PK: 5698
Part PK: 5699
Part PK: 5700
Part PK: 5701
Part PK: 5702
Part PK: 5703
Part PK: 5704
Part PK: 5712
Part PK: 5713
Part PK: 5778
Part PK: 5779


In [ ]:
for part in parts_without_regions:
    delete_part(555,part)